
#### **Ejercicios1 - Participación LLMs & NLP (torchtext + Hugging Face)**

Cuaderno de **6 ejercicios cortos** (10-15 min c/u) que refuerzan:
- *Introduccion_LLMs_y_Caso_de_Uso_Torchtext_HF* (torchtext + flujo clásico)
- *Librerias_NLP* (panorama y Transformers de Hugging Face)

> **Nota**: La primera ejecución puede descargar pesos de modelos. Mantén registro de versiones (`torch`, `transformers`, `torchtext`) para reproducibilidad.


In [ ]:
# Configuración de hardware y caché de Hugging Face (robusta)
# Verifica GPU/CPU y configura caché local sin usar TRANSFORMERS_CACHE (deprecado).
import os, sys, platform, warnings
from pathlib import Path

# (1) Caché local recomendado
CACHE_BASE = Path("./hf_cache").resolve()
os.environ["HF_HOME"] = str(CACHE_BASE)                 # único root recomendado
os.environ["HUGGINGFACE_HUB_CACHE"] = str(CACHE_BASE / "hub")  # opcional
for p in (CACHE_BASE, CACHE_BASE / "hub"):
    p.mkdir(parents=True, exist_ok=True)

# Si alguien dejó TRANSFORMERS_CACHE en el entorno, lo quitamos para evitar FutureWarning
if "TRANSFORMERS_CACHE" in os.environ:
    os.environ.pop("TRANSFORMERS_CACHE", None)
    print("TRANSFORMERS_CACHE eliminado del entorno para evitar deprecación.")

print("HF_HOME:", os.environ.get("HF_HOME"))
print("HUGGINGFACE_HUB_CACHE:", os.environ.get("HUGGINGFACE_HUB_CACHE"))

# (2) Activar descargas rápidas solo si 'hf_transfer' está instalado
try:
    import hf_transfer  # noqa: F401
    os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
    print("HF_HUB_ENABLE_HF_TRANSFER: 1 (hf_transfer disponible)")
except Exception:
    os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"
    print("HF_HUB_ENABLE_HF_TRANSFER: 0 (hf_transfer no instalado)")

# (3) Verificar dispositivo disponible
try:
    import torch
    if torch.cuda.is_available():
        idx = torch.cuda.current_device()
        device = f"cuda:{idx}"
        print("CUDA disponible:", True)
        print("Cantidad de dispositivos CUDA:", torch.cuda.device_count())
        print("Nombre del dispositivo actual:", torch.cuda.get_device_name(idx))
        print("Capacidad de CUDA:", torch.cuda.get_device_capability(idx))
    elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
        device = "mps"
        print("Apple MPS disponible:", True)
    else:
        device = "cpu"
        print("GPU no disponible; usando CPU.")
    print("Dispositivo seleccionado:", device)
except Exception as e:
    print("Torch no disponible o la consulta de dispositivo falló:", e)

# (4) Silenciar advertencias deprecadas de torchtext 0.18
try:
    import torchtext
    torchtext.disable_torchtext_deprecation_warning()
    print("Advertencias deprecadas de torchtext silenciadas.")
except Exception as e:
    print("No se pudo silenciar la advertencia de torchtext:", e)

# (5) Información del entorno
print("Python:", sys.version.split()[0], "| Plataforma:", platform.platform())


In [ ]:
# Configuración de hardware y caché de Hugging Face (robusta)
# Verifica GPU/CPU y configura caché local sin usar TRANSFORMERS_CACHE (deprecado).
import os, sys, platform, warnings
from pathlib import Path

# (1) Caché local recomendado
CACHE_BASE = Path("./hf_cache").resolve()
os.environ["HF_HOME"] = str(CACHE_BASE)                 # único root recomendado
os.environ["HUGGINGFACE_HUB_CACHE"] = str(CACHE_BASE / "hub")  # opcional
for p in (CACHE_BASE, CACHE_BASE / "hub"):
    p.mkdir(parents=True, exist_ok=True)

# Si alguien dejó TRANSFORMERS_CACHE en el entorno, lo quitamos para evitar FutureWarning
if "TRANSFORMERS_CACHE" in os.environ:
    os.environ.pop("TRANSFORMERS_CACHE", None)
    print("TRANSFORMERS_CACHE eliminado del entorno para evitar deprecación.")

print("HF_HOME:", os.environ.get("HF_HOME"))
print("HUGGINGFACE_HUB_CACHE:", os.environ.get("HUGGINGFACE_HUB_CACHE"))

# (2) Activar descargas rápidas solo si 'hf_transfer' está instalado
try:
    import hf_transfer  # noqa: F401
    os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
    print("HF_HUB_ENABLE_HF_TRANSFER: 1 (hf_transfer disponible)")
except Exception:
    os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"
    print("HF_HUB_ENABLE_HF_TRANSFER: 0 (hf_transfer no instalado)")

# (3) Verificar dispositivo disponible
try:
    import torch
    if torch.cuda.is_available():
        idx = torch.cuda.current_device()
        device = f"cuda:{idx}"
        print("CUDA disponible:", True)
        print("Cantidad de dispositivos CUDA:", torch.cuda.device_count())
        print("Nombre del dispositivo actual:", torch.cuda.get_device_name(idx))
        print("Capacidad de CUDA:", torch.cuda.get_device_capability(idx))
    elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
        device = "mps"
        print("Apple MPS disponible:", True)
    else:
        device = "cpu"
        print("GPU no disponible; usando CPU.")
    print("Dispositivo seleccionado:", device)
except Exception as e:
    print("Torch no disponible o la consulta de dispositivo falló:", e)

# (4) Silenciar advertencias deprecadas de torchtext 0.18
try:
    import torchtext
    torchtext.disable_torchtext_deprecation_warning()
    print("Advertencias deprecadas de torchtext silenciadas.")
except Exception as e:
    print("No se pudo silenciar la advertencia de torchtext:", e)

# (5) Información del entorno
print("Python:", sys.version.split()[0], "| Plataforma:", platform.platform())



#### **Ejercicio 1 - Mini-pipeline "clásico" con `EmbeddingBag` (torchtext)**

**Objetivo:** tokenización -> vocabulario -> ids -> `EmbeddingBag` (sin entrenar).  
**Entrega:** explica brevemente qué representan `ids`, `offsets` y por qué `out` es `(3, 16)`. Incluye un print del tamaño del vocabulario (`len(vocab.get_stoi())`).


In [ ]:
# Objetivo: pipeline clásico mínimo con vocabulario -> ids -> EmbeddingBag
import torch
from torch import nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Conjunto de textos de ejemplo
texts = [
    "devops pipelines are reliable",
    "security gates catch issues",
    "infra as code is reproducible",
]

# Tokenizador básico en inglés
tok = get_tokenizer("basic_english")

# Función generadora de tokens
def yield_tokens(data):
    for t in data:
        yield tok(t)

# Construcción de vocabulario con un token especial <unk>
vocab = build_vocab_from_iterator(yield_tokens(texts), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Conversión de textos a IDs (tensores de enteros)
ids = [torch.tensor(vocab(tok(t)), dtype=torch.long) for t in texts]

# Offsets para EmbeddingBag (marca dónde empieza cada frase)
offsets = torch.tensor([0, len(ids[0]), len(ids[0]) + len(ids[1])])

# Concatenación de todos los IDs en un solo tensor
flat = torch.cat(ids)

# Capa de embeddings con modo "mean" (media de embeddings por oración)
emb = nn.EmbeddingBag(num_embeddings=len(vocab), embedding_dim=16, mode="mean")
out = emb(flat, offsets)

# Salidas de depuración
print("Tamaño stoi (vocabulario):", len(vocab.get_stoi()))
print("Longitudes de ids:", [len(x) for x in ids])
print("Offsets:", offsets.tolist())
print("Forma de salida:", tuple(out.shape))  # esperado (3, 16)


In [ ]:
# Celda para completar
# 1) En 3-4 líneas, explica qué representan 'ids' y 'offsets'.
#    - 'ids' contiene los tensores con los índices de vocabulario para cada oración.
#    - 'offsets' indica las posiciones iniciales de cada oración dentro del tensor concatenado.
# 2) ¿Por qué la salida (Out) tiene forma (3, 16)?
#    - Porque hay 3 oraciones de entrada y cada una se proyecta a un embedding de dimensión 16.
# 3) Imprime las primeras 5 entradas de vocab.get_stoi() como evidencia.
#    PISTA: list(vocab.get_stoi().items())[:5]
# TODO: agrega aquí tu explicación y prints.



#### **Ejercicio 2 - Comparación de tokenización HF (BERT-m vs XLM-R)**

**Objetivo:** observar diferencias de *subwords* en español.  
**Entrega:** tabla breve (palabra -> fragmentación por modelo) y 3-4 líneas sobre impacto en longitud de secuencia/memoria.


In [ ]:
# Objetivo: compara segmentacion de subpalabras en español
from transformers import AutoTokenizer

# Texto de ejemplo en español sobre Nginx
text = "Configurar Nginx como reverse proxy mejora seguridad y rendimiento del servicio."

# Cargamos dos tokenizadores distintos
t1 = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
t2 = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Tokenización con cada modelo
tokens1 = t1.tokenize(text)  # WordPiece (## para subpalabras)
tokens2 = t2.tokenize(text)  # SentencePiece (▁ para inicio de palabra)

# Mostramos resultados y longitudes
print("Tokens BERT-m:", tokens1, "len:", len(tokens1))
print("Tokens XLM-R:", tokens2, "len:", len(tokens2))



In [ ]:
# Celda para completar
# Construye una pequeña tabla comparativa para 5-7 palabras interesantes (manual o programática).
# Luego escribe 3-4 líneas (en español) sobre cómo las diferencias de subpalabras impactan
# en la longitud de la secuencia y en el uso de memoria.
# CONSEJO: elige palabras como "Configurar", "Nginx", "rendimiento", "seguridad", "servicio".
# TODO: tu análisis / tabla aquí.



#### **Ejercicio 3: Análisis de sentimiento multilingüe con `pipeline`**

**Objetivo:** ejecutar un *pipeline* listo para usar y contar etiquetas.  
**Entrega:** `Counter` de etiquetas y 2-3 líneas sobre coincidencias/discrepancias con tu intuición.


In [ ]:
# Objetivo: inferencia rápida y conteo de etiquetas
from transformers import pipeline
from collections import Counter

# Definimos un pipeline de análisis de sentimiento multilingüe
clf = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# Textos de prueba en diferentes idiomas
texts = [
    "Excelente experiencia con el despliegue.",
    "El build rompió producción, muy mal.",
    "Service is okay, not great.",
    "La latencia bajó notablemente, bien.",
    "Terrible soporte al cliente.",
]

# Ejecutamos la inferencia sobre los textos
res = clf(texts)

# Contamos cuántas veces aparece cada etiqueta
cnt = Counter([r["label"] for r in res])

# Mostramos resultados brutos y conteo de etiquetas
print("Resultados brutos:", res)
print("Conteo de etiquetas:", cnt)


In [ ]:
# Celda para completar
# Escribe 2-3 líneas comentando si los resultados coinciden con tu intuición.
# Si hay discrepancias, da un ejemplo y una posible explicación.
# TODO: tu comentario aquí.



#### **Ejercicio 4- Zero-shot para moderación ligera (MNLI)**

**Objetivo:** etiquetar textos sin *finetuning* con `zero-shot-classification`.  
**Entrega:** tabla (texto -> predicción -> score) + tu "ground truth" y un % de acierto.


In [ ]:
# Objetivo: boceto rápido de moderación con zero-shot
from transformers import pipeline

# Definimos un pipeline de clasificación zero-shot con BART MNLI
zsc = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Posibles etiquetas de moderación
labels = ["toxicity", "sexual", "violence", "insult", "safe"]

# Textos de prueba en español e inglés
texts = [
    "Buen trabajo en el despliegue.",
    "Eres un inútil total.",
    "El informe incluye fotos violentas.",
    "Contenido educativo para niños.",
    "Comentarios groseros y ofensivos.",
    "This is purely technical documentation.",
    "Amenazas explícitas contra usuarios.",
    "Chistes subidos de tono.",
]

# Inferencia: seleccionamos la etiqueta más probable para cada texto
pred = []
for t in texts:
    r = zsc(t, labels)
    pred.append((t, r["labels"][0], round(r["scores"][0], 3)))

# Mostramos los resultados (texto, etiqueta asignada, puntuación)
for p in pred:
    print(p)


In [ ]:
# Celda  para completar
# 1) Define tu lista manual de ground truth para los textos anteriores (elige la mejor clase por fila).
# 2) Calcula la exactitud simple = correctos / 8.
# 3) Muestra una tabla de 4 columnas: texto | predicción | puntaje | ground_truth | ¿correcto?
# PISTA: construye una lista de diccionarios e imprime de forma legible (o usa pandas si está disponible).
# TODO: tu evaluación aquí.



#### **Ejercicio 5:  Resumen y control de longitud (DistilBART)**

**Objetivo:** resumir un párrafo técnico y medir compresión.  
**Entrega:** tabla con longitudes y *compression ratio (relación de comprensión)* + 3 viñetas sobre qué se gana/pierde al variar `max_length`.


In [ ]:
# Resumen con DistilBART usando safetensors (evita torch.load de .bin)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_id = "sshleifer/distilbart-cnn-12-6"

tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
modelo = AutoModelForSeq2SeqLM.from_pretrained(model_id, use_safetensors=True)

summ = pipeline("summarization", model=modelo, tokenizer=tok)

text = (
  "Implementar despliegues canarios reduce el riesgo al liberar cambios a un pequeño "
  "porcentaje de usuarios. Combinado con métricas de latencia, error rate y saturación, "
  "permite decisiones informadas sobre promoción o rollback. La observabilidad con trazas "
  "distribuidas ayuda a localizar cuellos de botella. Feature flags facilitan activar o "
  "desactivar funciones sin redeploy. La automatización en CI/CD asegura reproducibilidad "
  "y tiempos de entrega más cortos, manteniendo la seguridad mediante escáneres en el pipeline."
)

s1 = summ(text, max_length=80, min_length=30, do_sample=False)[0]["summary_text"]
s2 = summ(text, max_length=120, min_length=30, do_sample=False)[0]["summary_text"]

def ratio(s, t): 
    return round(len(s)/len(t), 3)

print("Longitud original:", len(text))
print("Longitud s1:", len(s1), "relación:", ratio(s1, text))
print("Longitud s2:", len(s2), "relación:", ratio(s2, text))
print("S1:", s1, "\nS2:", s2)


In [ ]:
# Celda para completar
# 1) Crea una pequeña tabla: [longitud_original, long_s1, long_s2, ratio_s1, ratio_s2].
# 2) Escribe 3 viñetas (en español) describiendo los compromisos al aumentar max_length.
# TODO: tu tabla y viñetas aquí.


#### **Ejercicio 6 -Comparación de tokenizadores en español**

El objetivo es observar cómo distintos modelos de HuggingFace tokenizan un mismo texto en español y analizar las diferencias en los sub-tokens generados.

In [ ]:
from transformers import AutoTokenizer

# Modelos a comparar
models = [
    "bert-base-multilingual-cased",
    "xlm-roberta-base"
]

text = "La observabilidad en DevOps ayuda a detectar cuellos de botella en sistemas distribuidos."

# TODO: itera sobre la lista de modelos, carga el tokenizer y muestra los tokens
# Usa print para visualizar las diferencias


In [ ]:
# Celda para completar
# 1) Muestra el número de tokens generados por cada modelo.
# 2) Compara cómo se segmenta la palabra "observabilidad".
# 3) Escribe 3 viñetas (en español) sobre las ventajas y limitaciones de cada tokenizador.
# TODO: tus resultados aquí



#### **Recomendaciones de entrega**
- Estructura de carpetas: `Participacion_LLMs_NLP` con `README.md` (hallazgos) y `out/` (salidas).
- Documenta versiones y hardware; conserva logs de consola relevantes.
- Si hay descargas lentas, precarga el modelo antes de la actividad grupal.
